In [1]:
import sys
sys.path.append("../")

from pathlib import Path
import wandb
import shortuuid

from confection import Config
from desci_sense.configs import init_config

In [3]:
import pandas as pd

a = {"a": 1, "b": 2}
b = {"a": 2, "b": 3}
df = pd.DataFrame([a,b])

# get_data(df)

In [23]:
list(df.columns)

['a', 'b', 'c']

In [ ]:
data = []
for i, row in df.iterrows():
    data.append(list(row))

In [21]:
data = [list(row) for i, row in df.iterrows()]
data

[[1.0, 2.0, nan], [2.0, nan, 3.0]]

In [20]:
l = list(df.iterrows())
row = l[0][1]
list(row)

[1.0, 2.0, nan]

In [16]:
dd = list(df.itertuples())[0]

In [18]:
list(df.iloc[0])

[1.0, 2.0, nan]

In [ ]:
dict

In [2]:
def log_pred_wandb(wandb_run, result):

    # get a unique ID for this prediction
    pred_uid = shortuuid.ShortUUID().random(length=8)

    artifact = wandb.Artifact(f"pred_{wandb_run.id}_{pred_uid}", type="prediction")

    columns = ["User", "URL", "Text", "Reasoning Steps", "Predicted Label", "True Label", "Tweet ID"]

    # extract relevant columns from prediction
    pred_row = [
        result['tweet']['user_name'],
        result['tweet']['tweetURL'],
        result['tweet']['text'],
        result['answer']['reasoning'],
        result['answer']['final_answer'],
        "", # no gold label meanwhile
        result['tweet']['tweetID']
    ]
    data = [pred_row]

    # add data to table
    table =  wandb.Table(data=data, columns=columns)
    artifact.add(table, "predictions")

    # log immediately since we don't know when user will close the session
    wandb.log_artifact(artifact)
    

    

    


In [3]:
config = init_config()

In [4]:
print(config.to_str())

[model]
model_name = "mistralai/mistral-7b-instruct"
temperature = 0.6

[prompt]
template_path = "desci_sense/prompting/templates/p2.txt"

[wandb]
wand_entity = "common-sense-makers"
project = "st-demo"
wandb_db_name = "test-DB"


In [5]:
api = wandb.Api()

TYPE = "prediction"
PROJECT = "st_demo-v0.2"

artifacts = []
collections = [
    coll for coll in api.artifact_type(type_name=TYPE, project=PROJECT).collections()
]


for coll in collections:
    for artifact in coll.versions():
        artifacts += [artifact]


In [6]:
len(artifacts)

16

In [7]:
artifact = artifacts[0]

In [8]:
artifact

<Artifact QXJ0aWZhY3Q6NjMxOTAyMTA0>

In [9]:
dd = artifact.download()

wandb:   1 of 1 files downloaded.  


In [10]:
dd

'./artifacts/pred_rhjdpsuk_GmRyKBUV:v0'

In [11]:
table_path = f"{dd}/predictions.table.json"

In [12]:
import pandas as pd

In [13]:
import json
raw_data = json.load(Path(table_path).open())

In [14]:
raw_data["data"]

[['Nathan Schneider',
  'https://twitter.com/ntnsndr/status/1721589953509953620',
  "In case you're not on the fediverse, here's a thread I just did on the ongoing WeWork collapse and why (surprise surprise) it should have been a co-op: https://social.coop/@ntnsndr/111364589153380497 #exittocommunity",
  "[Reasoning Steps]\n\n1. The post is about the ongoing collapse of WeWork.\n2. The author is discussing the reasons for the collapse.\n3. The author suggests that the collapse should have been a cooperative.\n4. The author is advocating for a different model of business.\n5. The author is sharing their thoughts and opinions on a current event.\n\n[Candidate Tags]\n\n1. <event>: This tag is suitable because the post is about a current event, specifically the collapse of WeWork.\n2. <review>: This tag is not suitable because the post is not a review of a reference.\n3. <announce>: This tag is not suitable because the post is not an announcement of new research.\n4. <reading>: This tag is

In [16]:
raw_data["columns"]

['User',
 'URL',
 'Text',
 'Reasoning Steps',
 'Predicted Label',
 'True Label',
 'Name of Label Provider',
 'Tweet ID']

In [19]:
from tqdm import tqdm

In [40]:
rows = []
cols = None
for artifact in tqdm(artifacts, total=len(artifacts)):
    a_path = artifact.download()
    table_path = Path(f"{a_path}/predictions.table.json")
    raw_data = json.load(table_path.open())
    if not cols:
        cols = raw_data["columns"]
    else:
        assert cols == raw_data["columns"]

    # add wandb name to identify each artifact    
    row = raw_data["data"][0] + [artifact.name]
    rows += [row]

# add column for wandb name
cols += ["wandb name"]
    




100%|██████████| 16/16 [00:02<00:00,  5.62it/s]


In [39]:
row

['John Tasioulas',
 'https://twitter.com/JTasioulas/status/1660582630641872898',
 'For a long time political philosophers ignored democracy in favour of building increasingly arcane theories of justice, or they saw justice as in conflict with democracy. This book by @dsallentess seeks to put democracy at the very heart of justice. Looking forward to reading. https://t.co/O9Ii3we0b8',
 '[Reasoning Steps]\n\n1. The post is about a book.\n2. The author of the book is mentioned.\n3. The book is about democracy and justice.\n4. The author is a political philosopher.\n5. The post is a positive endorsement of the book.\n\n[Candidate Tags]\n\n<review> - This tag is suitable because the post is a positive endorsement of the book.\n<announce> - This tag is not suitable because the post is not announcing a new research output.\n<job> - This tag is not suitable because the post is not about a job.\n<event> - This tag is not suitable because the post is not about an event.\n<reading> - This tag is 

In [36]:
cols

['User',
 'URL',
 'Text',
 'Reasoning Steps',
 'Predicted Label',
 'True Label',
 'Name of Label Provider',
 'Tweet ID',
 'wandb name']

In [41]:
pd.DataFrame(data=rows, columns=cols)

,User,URL,Text,Reasoning Steps,Predicted Label,True Label,Name of Label Provider,Tweet ID,wandb name
0,Nathan Schneider,https://twitter.com/ntnsndr/status/17215899535...,"In case you're not on the fediverse, here's a ...",[Reasoning Steps]\n\n1. The post is about the ...,<event>,<other>,rtk,1721589953509953620,pred_rhjdpsuk_GmRyKBUV:v0
1,CU Boulder CMCI Media Studies,https://twitter.com/cumediastudies/status/1722...,We've got two scholars visiting on Thursday! K...,[Reasoning Steps]\n\n1. The post contains info...,<other>,<event>,rtk,1722066353316671536,pred_vwnrj1ks_3rnymiZi:v0
2,Herbert Chang 張賀鈞,https://twitter.com/herbschang/status/17220275...,QSS is hiring a postdoc @Dartmouth! (addn'l po...,[Reasoning Steps]\n\n1. The post contains info...,<job>,<job>,rtk,1722027569002811787,pred_o268mc8q_XVjWhDdp:v0
3,Nina Miolane,https://twitter.com/ninamiolane/status/1721901...,I am recruiting PhD students for 2024!😃\n\nYou...,[Reasoning Steps]\n\n1. The post contains a jo...,<job>,<job>,rtk,1721901767003419105,pred_3s7ax4lq_Kub7L2Ks:v0
4,Yogi Jaeger 💙 @yoginho@spore.social,https://twitter.com/yoginho/status/17207719711...,This take-down by @damiengwalter of Marc Andre...,[Reasoning Steps]\n\n1. The post discusses a c...,<review>,<review>,,1720771971149963679,pred_h2g8s66f_c4x3pbgJ:v0
5,Yohan J. John,https://twitter.com/DrYohanJohn/status/1721991...,Found a nice article on the origins of the nor...,[Reasoning Steps]\n\n1. The post contains a li...,<announce>,<review>,,1721991950772179418,pred_0jbmmgam_dsaToGUp:v0
6,Maria Repnikova,https://twitter.com/MariaRepnikova/status/1694...,"Looking forward to reading the piece, but leve...",[Reasoning Steps]\n\n1. The post discusses a p...,<other>,<reading>,rtk,1694054681025356286,pred_ozdgh4wy_jHQoK6kR:v0
7,Geoffrey Hinton,https://twitter.com/geoffreyhinton/status/1721...,Fei-Fei Li has written a book. She was the fir...,[Reasoning Steps]\n\n1. The post is about a bo...,<review>,<review>,rtk,1721937095860633663,pred_lzrlzyuq_V84u39A4:v0
8,Melanie Mitchell,https://twitter.com/MelMitchell1/status/170666...,“Embers of Autoregression”. \n\nLooking forwa...,[Reasoning Steps]\n\n1. The post is about a bo...,<reading>,<reading>,rtk,1706666878872248680,pred_1klkydec_LoXSsFSN:v0
9,Gaia Vince @wanderinggaia.bsky.social,https://twitter.com/WanderingGaia/status/14385...,Looking forward to reading this beautiful new ...,[Reasoning Steps]\n\n1. The post is about a bo...,reading,<reading>,rtk,1438525134478872583,pred_g8mxfgww_ZadGC6d8:v0


In [25]:
table_path.parent.name

'pred_kezzchu2_AfZd8RPg:v0'

In [32]:
artifact.name

'pred_kezzchu2_AfZd8RPg:v0'

In [42]:
pd.Categorical

pandas.core.arrays.categorical.Categorical

In [43]:
pd

<module 'pandas' from '/home/rkl25/anaconda3/envs/twscraping/lib/python3.11/site-packages/pandas/__init__.py'>